In [ ]:
import pandas as pd
from minio import Minio
from io import BytesIO


# Conexión con MinIO en localhost
client = Minio(
    "localhost:9000",  # Endpoint de MinIO en localhost
    access_key="minioadmin",  # Tu clave de acceso
    secret_key="minioadmin",  # Tu clave secreta
    secure=False  # No usar HTTPS para MinIO local
)

# Verificar si el bucket existe
if not client.bucket_exists("access-zone"):
    print("Bucket 'access-zone' no encontrado.")
else:
    # Leer el archivo Parquet desde el bucket
    try:
        # Descargar el archivo Parquet
        obj = client.get_object("access-zone", "analytics/congestion_by_hour.parquet")
        
        # Leer el archivo Parquet en Pandas
        df = pd.read_parquet(BytesIO(obj.read()))
    except Exception as e:
        print(f"Error al leer el archivo Parquet: {e}")




In [4]:
high_congestion_df = df[df['congestion_level'] == 'high']

# Mostrar las horas con mayor congestión y los vehículos predominantes
high_congestion_summary = high_congestion_df[['hour', 'congestion_level', 'vehiculo_predominante']]

high_congestion_summary



,station_origin_id,station_dest_id,user_type,total_viajes,avg_duration_seconds,avg_distance_km,total_users
0,1,20,occasional,1,919.0,2.53,1
1,1,25,annual,1,1038.0,2.85,1
2,1,34,annual,1,1403.0,3.86,1
3,2,29,annual,1,1357.0,3.73,1
4,2,33,annual,1,1107.0,3.04,1
...,...,...,...,...,...,...,...
138,42,13,annual,1,1113.0,3.06,1
139,42,25,annual,2,1193.5,3.28,2
140,42,35,annual,1,733.0,2.01,1
141,43,2,annual,1,1224.0,3.36,1


In [ ]:
# contar las ocurrencias de los tipos de vehículos predominantes en las horas de alta congestión:
vehicle_count = high_congestion_df['vehiculo_predominante'].value_counts()

print("\nTipos de vehículos predominantes en horarios de alta congestión:")
print(vehicle_count)




Tipos de vehículos predominantes en horarios de alta congestión:


vehiculo_predominante
cars    14
Name: count, dtype: int64

In [29]:
# Análisis de las franjas horarias con mayor congestión (se puede hacer por cantidad de vehículos, por ejemplo)
high_congestion_by_hour = high_congestion_df.groupby('hour').agg({'total_vehicles': 'sum'}).sort_values('total_vehicles', ascending=False)

print("\nFranjas horarias con mayor congestión total de vehículos:")
high_congestion_by_hour


Franjas horarias con mayor congestión total de vehículos:


,total_vehicles
hour,
8,9926
18,9330
9,8830
19,5688
12,4835
16,4622
11,4376
21,4144
22,3844
